In [ ]:
import os, sys
import holoviews as hv
sys.path.append(os.path.abspath('../../two_dim_majoranas/'))
import adaptive

adaptive.notebook_extension()
hv.notebook_extension('matplotlib')

import numpy as np
import scipy.constants
import cmath

import functools as ft

import sns_system, plotting_results
from distributed_sns import AggregatesSimulationSet as ASS
from distributed_sns import SimulationSet as SS

constants = dict(
    m_eff=0.023 * scipy.constants.m_e / (scipy.constants.eV * 1e-3) / 1e18,  # effective mass in kg, 
    hbar=scipy.constants.hbar / (scipy.constants.eV * 1e-3),
    e = scipy.constants.e,
    current_unit=scipy.constants.k * scipy.constants.e / scipy.constants.hbar * 1e9,  # to get nA
    mu_B=scipy.constants.physical_constants['Bohr magneton'][0] / (scipy.constants.eV * 1e-3),
    k=scipy.constants.k / (scipy.constants.eV * 1e-3),
    exp=cmath.exp,
    cos=cmath.cos,
    sin=cmath.sin
   )

import kwant
import matplotlib.pyplot as plt
import supercurrent_matsubara
from functools import partial

In [ ]:
syst_pars = {
    'L_down': 200,
     'L_up': 200,
     'L_m': 40,
     'L_x': 10,
     'a': 10,
     'mu_from_bottom_of_spin_orbit_bands': True,
     'with_vlead': True,
     'k_x_in_sc' : True}

params_raw= dict(g_factor_middle = 10,
                 g_factor_left = 0,
                 g_factor_right = 0,
                 mu = 2.0,
                 alpha_middle = 28,
                 alpha_left = 28,
                 alpha_right = 28,
                 Delta_left = 0.18,
                 Delta_right = 0.18,
                 B = 0.5,
                 phase = 0,
                 T =.1,
                 V=0)

params = dict(**constants,
              **params_raw)
_=plotting_results.plot_syst(syst_pars, params)

In [ ]:
import supercurrent
def cur(phase, L, Lm):
    import sns_system
    import supercurrent
    
    params['phase'] = phase
    syst_pars['L_up'] = L
    syst_pars['L_down'] = L
    syst_pars['L_m'] = Lm
    syst = sns_system.make_wrapped_system(**syst_pars)
    I_c2 = supercurrent.wrapped_current(syst_pars, params, syst_wrapped=syst, tol=1e-5)
    return 1e9 * I_c2

In [ ]:
for L in np.arange(100, 2000, 100):
    print(f'L={L} :\t', end='')
    for Lm in [40, 80, 120]:    
        print(cur(0.209439510239, L, Lm), end=', ')
    print('')

In [ ]:
def matsu_cur(Lx, Lm):  
    syst_pars['L_x'] = Lx
    syst_pars['L_m'] = Lm
    params['phase'] = 0

    syst, hopping = sns_system.make_sns_leaded_system(**syst_pars)

    el_blocks = partial(sns_system.take_electron_blocks, norbs=4)
    I_c = supercurrent_matsubara.I_c(syst, hopping, params, el_blocks, tol=1e-5)
    phases = I_c['phases']
    currents = I_c['currents']
    assert(0.209439510239 - 1e-6 < phases[16] < 0.209439510239 +1e-6)
    I=currents[16]
    return I/Lx

In [ ]:
for L in np.arange(100, 4000, 200):
    print(f'L={L} :\t', end='')
    for Lm in [40, 80, 120]:    
        print(matsu_cur(L, Lm), end='\t')
    
    print('')

In [ ]:
params

In [ ]:
1/(0.0889706851518/0.572495451563)

In [ ]:
from multiprocessing import Pool

syst = sns_system.make_wrapped_system(**syst_pars)

phases = np.linspace(-3, 3, 31)

p=Pool()
curs = np.array(p.map(cur, phases))

In [ ]:
hv.Curve((phases, curs))*tom_curve

In [ ]:
tom_curve = hv.Curve((phases, curs))
tom_curve

In [ ]:
tom_curve = hv.Curve((phases, curs))
tom_curve

In [ ]:
tom_curve = hv.Curve((phases, curs))
tom_curve

In [ ]:
mats_curve = hv.Curve((I_c['phases'], I_c['currents']/syst_pars['L_x']))
tom_curve*mats_curve

In [ ]:
%%time
params['phase'] = 0
el_blocks = partial(sns_system.take_electron_blocks, norbs=4)
I_c_leaded = supercurrent_matsubara.I_c(syst_leaded, hopping_leaded, params, el_blocks)

In [ ]:
# syst_pars['a'] = 1
syst_pars['L_x'] = 1000
syst_leaded, hopping_leaded = sns_system.make_sns_leaded_system(**syst_pars)
kwant.plot(syst_leaded);

In [ ]:
np.interp(.5, I_c_leaded['phases'], I_c_leaded['currents']/syst_pars['L_x'])

In [ ]:
%%time
params['phase'] = 0
el_blocks = partial(sns_system.take_electron_blocks, norbs=4)
I_c_leaded = supercurrent_matsubara.I_c(syst_leaded, hopping_leaded, params, el_blocks)

In [ ]:
np.interp(.5, I_c_leaded['phases'], I_c_leaded['currents']/syst_pars['L_x'])

In [ ]:
hv.Curve((phases, curs)) * hv.Scatter((I_c_leaded['phases'], I_c_leaded['currents']/600))

In [ ]:
hv.Curve((phases, curs/(15/np.pi))) * hv.Scatter((I_c_leaded['phases'], I_c_leaded['currents']/600))

In [ ]:
hv.Curve((phases, curs)) * hv.Scatter((I_c_leaded['phases'], I_c_leaded['currents']/syst_pars['L_x']))

In [ ]:
hv.Curve((phases, curs/curs.max())) * hv.Scatter((I_c['phases'], I_c['currents'] / I_c['currents'].max()))

In [ ]:
params['e'] / params['hbar']

In [ ]:
hv.Curve((phases, curs*15)) * hv.Scatter((I_c['phases'], I_c['currents'] / syst_pars['L_x']))

In [ ]:
hv.Curve((phases, curs/curs.max())) * hv.Scatter((I_c['phases'], I_c['currents'] / I_c['currents'].max()))

In [ ]:
curs = np.array(curs) - curs[15]
curs = curs/ curs.max()

In [ ]:
hv.Curve(curs[:15]) * hv.Curve(-curs[15:][::-1])

In [ ]:
c = I_c['currents'] / I_c['currents'].max()
hv.Curve(c[:15]) * hv.Curve(-c[15:][::-1])